# RAG Application in answering AIO Questions

## RAG without UI

### 1. Install necessary libs

In [2]:
!pip install -q transformers==4.41.2
!pip install -q bitsandbytes==0.43.1
!pip install -q accelerate==0.31.0
!pip install -q langchain==0.2.5
!pip install -q langchainhub==0.1.20
!pip install -q langchain-chroma==0.1.1
!pip install -q langchain-community==0.2.5
!pip install -q langchain_huggingface==0.0.3
!pip install -q python-dotenv==1.0.1
!pip install -q pypdf==4.2.0
!pip install -q numpy==1.24.4

### 2. Create vector database

#### 2.1 Import necessary libs

In [1]:
import torch

from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline

from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.chains import ConversationalRetrievalChain

from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub


#### 2.2 Read pdf file

In [2]:
Loader = PyPDFLoader
FILE_PATH = r"C:\Users\Admin\Downloads\mtt.pdf"
loader = Loader(FILE_PATH)
documents = loader.load()

In [3]:
documents

[Document(metadata={'source': 'C:\\Users\\Admin\\Downloads\\mtt.pdf', 'page': 0}, page_content='1 \n Chương I : Tổng quan về mạng máy tính  \n1.1. Giới thiệu về mạng máy tính  \n  Ngày nay mạng máy tính đã và đang được sử dụng rộng rãi trong hầu \nhết các lĩnh vực kinh tế, chính trị, xã hội không phải chỉ ở một quốc gia nào \nđó mà là trên toàn thế giới. Các mạng máy tính đã làm thay đổi một cách toàn \ndiện phương thức sống, làm việc và sinh hoạt của con người cũng như \nphương thức tổ chức và phát triển của xã hội. Nhịp độ xử dụng các mạng máy \ntính để trao đổi thông tin, xử lý thông tin trong cuộc sống và trong các t ổ chức \nngày càng tăng. Mạng máy tính chính là một trong những mục tiêu hướng tới \ncông nghiệp hóa, hiện đại hóa đất nước.  \n1.2. Khái niệm về mạng máy tính  \n  Định nghĩa : Mạng máy tính là mạng bao gồm ít nhất hai máy tính \nđược kết nối với nhau thông qua đường  truyền vật lý cho phép chúng chia sẻ \ndữ liệu và tài nguyên chung. Tất cả các hệ thống mạng máy tính

#### 2.3 Init text splitter and split file into chunks

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                               chunk_overlap=100)

docs = text_splitter.split_documents(documents)

print("Number of sub-documents: ", len(docs))
print(docs[0])

Number of sub-documents:  355
page_content='1 
 Chương I : Tổng quan về mạng máy tính  
1.1. Giới thiệu về mạng máy tính  
  Ngày nay mạng máy tính đã và đang được sử dụng rộng rãi trong hầu 
hết các lĩnh vực kinh tế, chính trị, xã hội không phải chỉ ở một quốc gia nào 
đó mà là trên toàn thế giới. Các mạng máy tính đã làm thay đổi một cách toàn 
diện phương thức sống, làm việc và sinh hoạt của con người cũng như 
phương thức tổ chức và phát triển của xã hội. Nhịp độ xử dụng các mạng máy 
tính để trao đổi thông tin, xử lý thông tin trong cuộc sống và trong các t ổ chức 
ngày càng tăng. Mạng máy tính chính là một trong những mục tiêu hướng tới 
công nghiệp hóa, hiện đại hóa đất nước.  
1.2. Khái niệm về mạng máy tính  
  Định nghĩa : Mạng máy tính là mạng bao gồm ít nhất hai máy tính 
được kết nối với nhau thông qua đường  truyền vật lý cho phép chúng chia sẻ 
dữ liệu và tài nguyên chung. Tất cả các hệ thống mạng máy tính lớn  đều xuất 
phát từ hệ thống đơn giản như vậy. ý tưởng để kết 

#### 2.4 Init instance vectorization

In [ ]:
embedding = HuggingFaceEmbeddings()

E:\anaconda3\envs\llama3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


#### 2.5 Init vector database

In [ ]:
vector_db = Chroma.from_documents(documents=docs,
                                  embedding=embedding)
retriever = vector_db.as_retriever()

#### 2.6 Try to query

In [6]:
result = retriever.invoke("Mạng máy tính là gì?")

print("Number of relevant documents: ", len(result))

Number of relevant documents:  4


### 3 Init LLMs

#### 3.1 Init params for model

In [7]:
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

#### 3.2 Init model and tokenizer

In [12]:
import torch
torch.__version__

'2.3.1+cpu'

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [8]:
MODEL_NAME = "lmsys/vicuna-7b-v1.5"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=nf4_config,
    low_cpu_mem_usage=True
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

d:\anaconda3\envs\ragenv\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--lmsys--vicuna-7b-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


RuntimeError: No GPU found. A GPU is needed for quantization.

#### 3.3 Integrate tokenizer and model into a pipeline

In [14]:
model_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id,
    device_map="auto"
)

llm = HuggingFacePipeline(
    pipeline=model_pipeline,
)


### 4 Run the program

In [15]:
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

USER_QUESTION = "YOLOv10 là gì?"
output = rag_chain.invoke(USER_QUESTION)
answer = output.split('Answer:')[1].strip()
print(answer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


YOLOv10 là một phiên bản của YOLO (You Only Look Once) - một hệ thống dự đoán hình ảnh được huấn luyện sẵn trên bộ dữ liệu COCO. Phiên bản này được tạo bằng cách tải về trọng số (file.pt) từ GitHub và khởi tạo mô hình bằng cách sử dụng thư viện ultralytics.
